In [5]:
import globals
import os
import pandas as pd

# Loads confusion matrices from each CV, calculates the mean and standard deviation
# and saves them to a file

paths = [os.path.join(globals.BASE_PATH, "experiments", n, f"conf_matrix_{n}.csv") for n in globals.RANDOM_INIT_EXP_NAMES]

conf_matrices = [pd.read_csv(path, index_col=0) for path in paths]

mean_conf_matrix = pd.concat(conf_matrices).groupby(level=0).mean()
std_conf_matrix = pd.concat(conf_matrices).groupby(level=0).std()

mean_conf_matrix

,benign,malignant,normal
benign,0.862791,0.077907,0.059302
malignant,0.117949,0.838462,0.043590
normal,0.119355,0.041935,0.838710


In [6]:
std_conf_matrix

,benign,malignant,normal
benign,0.035439,0.034254,0.021546
malignant,0.063040,0.069489,0.029731
normal,0.074574,0.034173,0.076033


In [7]:
mean_conf_matrix.to_csv("mean_conf_matrix.csv")
std_conf_matrix.to_csv("std_conf_matrix.csv")